## Pergunta de negócio: Como evoluíram as vendas mensais por categoria entre 2022 e 2025? 


In [0]:
from pyspark.sql.functions import sum, col, desc, row_number, round
import matplotlib.pyplot as plt

# Tabela fato_vendas 
fato_vendas = "data_ex.gold.fato_vendas"
dim_categoria    = "data_ex.gold.dim_categoria_produto"


# Seleção da categoria e de sua respectiva sk
categoria_produto = (
    spark.table(dim_categoria)
         .select("categoria_nome", "sk_categoria")
         .distinct()
)

# Seleção de vendas totais por categoria em função de ano/mes
vendas = (spark.table(fato_vendas)
                    .groupBy("ano", "mes", "sk_categoria")
                    .agg(sum("valor_total").alias("total_vendas"))  
                )

vendas = (vendas.join(categoria_produto.select("sk_categoria", "categoria_nome"), on="sk_categoria").drop("sk_categoria"))

# Separacao por categoria
for categoria in categoria_produto.select("categoria_nome").collect():
    
    vendas_categoria = (
        vendas
            .filter(col("categoria_nome") == categoria.categoria_nome)
            .orderBy("ano", "mes")
            .select("ano", "mes", "total_vendas")
            .collect()
    )
    
    if not vendas_categoria:
        continue

    x = [f"{r.ano}-{str(r.mes).zfill(2)}" for r in vendas_categoria]
    y = [r.total_vendas for r in vendas_categoria]

    # Plot
    plt.figure(figsize=(18, 4))
    plt.plot(x, y, marker="o")
    plt.title(f"Vendas - {categoria.categoria_nome}")
    plt.xlabel("Período (Ano-Mês)")
    plt.ylabel("Total de Vendas")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()


## Pergunta de negócio: Quais os top 10 produtos por valor_total e sua participação no total? 
* Consulta: ranking por valor_total com cálculo de participação (%) sobre o total geral.
* Insight esperado: Insight: concentração de receita e dependência de poucos produtos.
 

In [0]:
from pyspark.sql.window import Window


# Participação entre 2022 há 2025
vendas_ciclo = ( vendas.groupBy("categoria_nome")
                .agg(sum("total_vendas").alias("total_vendas_ciclo"))
                )

vendas_totais = vendas_ciclo.agg(sum("total_vendas_ciclo")).first()[0]

vendas_rank = (
    vendas_ciclo
        .withColumn("rank", row_number().over(Window.orderBy(col("total_vendas_ciclo").desc())))
)
vendas_rank.show()

top_10 = vendas_rank.filter(col("rank") <= 10)
x = top_10.withColumn("participacao [%]", round((col("total_vendas_ciclo") * 100) / vendas_totais, 4) )
x.show()




## Pergunta de negócio: Quais localidades (UF/cidade) apresentam maior ticket médio?
* Consulta: média de valor_total por pedido (ou por cliente) agrupada por UF/cidade.
* Insight esperado: Insight: potenciais mercados premium e oportunides regionais.

In [0]:
dim_localidade = "data_ex.gold.dim_localidade"

localidades = spark.table(dim_localidade).select("cidade_venda", "estado_venda", "sk_localidade")

vendas_parciais = (spark.table(fato_vendas)
                    .filter(col("sk_localidade").isNotNull())
                    .groupBy("sk_localidade")
                    .agg(sum("valor_total").alias("total_vendas_localidade"))  
                  )

vendas_parciais = vendas_parciais.join(localidades, on="sk_localidade", how="inner").drop("sk_localidade")

vendas_totais = vendas_parciais.agg(sum("total_vendas_localidade").alias("vendas")).first()["vendas"]

media_vendas = vendas_parciais\
                  .withColumn("media_vendas", round(col("total_vendas_localidade")/vendas_totais,6))\
                  .withColumn("participacao[%]", round((col("total_vendas_localidade")*100)/vendas_totais,6))

maiores_tickets = (
    media_vendas
        .withColumn("rank", row_number().over(Window.orderBy(col("media_vendas").desc())))
)

maiores_tickets_localidade = maiores_tickets.filter(col("rank") <= 20)

maiores_tickets_localidade.show()

estados = media_vendas.groupBy("estado_venda").agg(sum("participacao[%]"))
estados = estados.withColumn("participacao[%]", round(col("sum(participacao[%])"),6)).drop("sum(participacao[%])")
estados_tickets = estados.withColumn("rank", row_number().over(Window.orderBy(col("participacao[%]").desc())))
estados_tickets = estados_tickets.filter(col("rank") <= 20)
estados_tickets.show()

## Pergunta de negócio: Como a quantidade média por transação varia por categoria e por ano?
* Consulta: média de quantidade por categoria_id e por ano(data_id). 
* Insight esperado: Insight: mudanças de comportamento de compra (packs maiores/menores).

##  Pergunta de negócio: Quais clientes são responsáveis por 80% do faturamento (curva ABC)?
* Consulta: cálculo de valor_total por cliente, ordenação e cumulativo até 80%. 
* Insight esperado: Insight: segmentação de clientes estratégicos para ações de retenção